El objetivo de este notebook es plotear y analizar tiempos / spikes de experimentos. Por un lado cargamos un archivo donde se indica el tiempo de activación del láser, y por otro tenemos los spikes. Tenemos que plotear primero, de manera coordinada, la información de ambos archivos.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import time
import datetime as dt
import os
import seaborn as sns
# comment2

In [ ]:
hoy = dt.date.today()

In [ ]:
df_times = pd.read_csv('tiempos.csv', sep=', ', header=None, index_col=0)
df_times

In [ ]:
exp_date = '2024_02_20'

name_sample = 'D19_POSTsiembra_A1_(I)(001)'
name_file_spikes = f'{name_sample}_spike_list.csv'
name_file_laser = '202402020_MEAs_A1_(I).xls'

timestamp = [int(i) for i in df_times[1].loc[name_sample].split(':')]

time_absolute_start = dt.datetime.combine(hoy, dt.time(timestamp[0], timestamp[1], timestamp[2], timestamp[3] * 1000))  # A1_(IV)_(000)

os.makedirs(f'../../results/{exp_date}', exist_ok=True)


time_absolute_start

In [ ]:
df_spikes = pd.read_csv(f"../../spike_lists/{exp_date}/{name_file_spikes}", sep='\t', encoding='unicode_escape')
df_spikes['electrode_str'] = [i.split('_')[1] for i in df_spikes['Electrode']]
df_spikes['electrode_coord'] = [(int(electrode[0]) - 1, 3 - (int(electrode[1]) - 1)) for electrode in df_spikes['electrode_str']]
df_spikes

In [ ]:
df_laser = pd.read_excel(f"../../tiempos_structura/{exp_date}/{name_file_laser}", index_col=0)
df_laser['start_times'] = [(dt.datetime.combine(hoy, df_laser['Turn ON Laser Time'].iloc[i]) - time_absolute_start).total_seconds() for i in range(len(df_laser))]
df_laser['end_times'] = [(dt.datetime.combine(hoy, df_laser['Turn OFF Laser Time'].iloc[i]) - time_absolute_start).total_seconds() for i in range(len(df_laser))]
df_laser['duration'] = df_laser['ON duration (ms)'] / 1000
df_laser


In [ ]:
voltages = sorted(set(df_laser['Laser Power (mW)'].values))
electrodes = sorted(set(df_spikes['electrode_str'].values))

window = 2

fig, axs = plt.subplots(len(voltages), 1, figsize=(25, 18))


for idx, voltage in enumerate(voltages):
    df_laser_sub = df_laser[df_laser['Laser Power (mW)'] == voltage]
    t_0, t_f = min(df_laser_sub['start_times']), max(df_laser_sub['end_times'])


    for rep, start, end in zip(df_laser_sub['Repetition'].values, df_laser_sub['start_times'].values, df_laser_sub['end_times'].values):
        axs[idx].add_patch(mpl.patches.Rectangle((start, -1), end - start, len(electrodes)+1, alpha=0.25))

    for electrode_idx, electrode in enumerate(electrodes):
        df_spikes_sub = df_spikes[(df_spikes['Time (s)'] > t_0 - window) & (df_spikes['Time (s)'] < t_f + window) & (df_spikes['electrode_str'] == electrode)]
        axs[idx].scatter(x=df_spikes_sub['Time (s)'].values, y=[electrode_idx] * len(df_spikes_sub), marker='|', c='#455454')
    


    axs[idx].set_yticks(np.array(range(len(electrodes))))
    axs[idx].set_yticklabels(electrodes)
    axs[idx].set_ylim([-1, len(electrodes)])

plt.savefig(f'../../results/{exp_date}/{name_sample}.png', dpi=300)

# Calculating the mean representation time between pulses

In [ ]:
tw = 0.1

time_ranges = np.arange(np.min(df_laser['start_times']) , np.max(df_laser['end_times']) , tw)
time_ranges_mean = (time_ranges[1:] + time_ranges[:-1]) * 0.5 

counts = np.array([len(df_spikes[(df_spikes['Time (s)'] >= start) & (df_spikes['Time (s)'] < end)]) for start, end in zip(time_ranges[:-1], time_ranges[1: ])])
len(counts)


In [ ]:
tmps = np.argwhere(counts > 80).flatten() * tw
diff = tmps[1: ] - tmps[: -1]
sns.swarmplot(diff[diff > 2 * tw])

